### Import and config

In [1]:
# Imports
import os
import time
import random
import json
import logging

from dotenv import load_dotenv
from tqdm import tqdm

import pandas as pd
import requests

from sqlalchemy import create_engine, text, inspect, Text, Integer, BigInteger, Float, Boolean
from sqlalchemy.dialects.postgresql import JSONB

# Configuration
load_dotenv()

# API urls
AUTH_URL = os.getenv('AUTH_URL')
ACTIVITIES_URL = os.getenv('ACTIVITIES_URL')
ACTIVITY_DETAIL_URL_TMPL = os.getenv('ACTIVITY_DETAIL_URL_TMPL')
ACTIVITY_KUDOS_URL_TMPL = os.getenv('ACTIVITY_KUDOS_URL_TMPL')
ACTIVITY_ZONES_URL_TMPL = os.getenv('ACTIVITY_ZONES_URL_TMPL')

# API keys
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

# DB
DB_URI = os.getenv('DB_URI')

# Bronze tables
TARGET_B_SCHEMA = os.getenv('TARGET_B_SCHEMA')
ACTIVITIES_B_TABLE = os.getenv('ACTIVITIES_B_TABLE')
DETAILS_B_TABLE = os.getenv('DETAILS_B_TABLE')
KUDOS_B_TABLE = os.getenv('KUDOS_B_TABLE')
ZONES_B_TABLE = os.getenv('ZONES_B_TABLE')

# Pagination
PER_PAGE = int(os.getenv('PER_PAGE'))
MAX_PAGES = int(os.getenv('MAX_PAGES'))

# Timeouts and retries
REQUEST_TIMEOUT = int(os.getenv('REQUEST_TIMEOUT'))
MAX_RETRIES = int(os.getenv('MAX_RETRIES'))
BASE_SLEEP = float(os.getenv('BASE_SLEEP'))

REFRESH_THRESHOLD_DAYS = int(os.getenv('REFRESH_THRESHOLD_DAYS'))

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

pd.set_option('display.max_columns', None)

### API keys validation

In [2]:
REQUIRED_API_ENV = ['CLIENT_ID', 'CLIENT_SECRET', 'REFRESH_TOKEN']
missing_api_env = [env for env in REQUIRED_API_ENV if not os.getenv(env)]
if missing_api_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_api_env)}.")

### DB names validation

In [3]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_B_SCHEMA', 'ACTIVITIES_B_TABLE', 'DETAILS_B_TABLE', 'ZONES_B_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Connecting with PostgreSQL

In [4]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
with engine.connect() as conn:
  conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {TARGET_B_SCHEMA};"))
logging.info(f"Connection established and {TARGET_B_SCHEMA} schema exists.")

2025-10-08 10:00:43,575 | INFO | Connection established and bronze schema exists.


### Token authorization

In [5]:
def get_access_token(auth_url: str, client_id: str, client_secret: str, refresh_token: str, timeout: int = 30) -> str:
  """
  Request a new access token from an OAuth2 authentication endpoint.

  The function uses a refresh token to obtain a short-lived access token. 
  If the response does not contain an ``access_token``, a RuntimeError is raised.

  Parameters
  ----------
  auth_url : str
      URL of the OAuth2 token endpoint.
  client_id : str
      OAuth2 client identifier.
  client_secret : str
      OAuth2 client secret.
  refresh_token : str
      Refresh token used to request a new access token.
  timeout : int, default=30
      Timeout in seconds for the HTTP request.

  Returns
  -------
  str
      The access token string retrieved from the authentication server.

  Raises
  ------
  RuntimeError
      If the response does not include an ``access_token``.
  requests.exceptions.RequestException
      If the HTTP request fails (e.g., network error, timeout).

  Notes
  -----
  Logs an informational message when the token is successfully retrieved.
  """

  payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': 'refresh_token',
  }
  res = requests.post(auth_url, data=payload, timeout=timeout)
  data = res.json()
  token = data.get('access_token')
  if not token:
    raise RuntimeError(f"No access token in response: {data}")
  logging.info('Access token retrived.')
  return token

access_token = get_access_token(AUTH_URL, CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)

2025-10-08 10:00:43,843 | INFO | Access token retrived.


### HTTP session

In [6]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {access_token}"})

def get_json_with_retry(url: str, params=None, max_retries: int = MAX_RETRIES, timeout: int = REQUEST_TIMEOUT, base_sleep: float = BASE_SLEEP):
  """
  Send a GET request with automatic retries and return the parsed JSON response.

  The function retries on:
    * HTTP 429 (rate limiting) — respects the ``Retry-After`` header if present,
      otherwise waits an increasing backoff time.
    * HTTP 5xx errors — retries with exponential backoff and jitter.
    * Network/connection errors — retries with exponential backoff and jitter.

  For each attempt, the backoff time increases by ``base_sleep * attempt`` seconds
  plus a small random jitter. On the final attempt, any error is raised.

  Parameters
  ----------
  url : str
      Endpoint URL to send the GET request to.
  params : dict, optional
      Query string parameters to include in the request.
  max_retries : int, default=MAX_RETRIES
      Maximum number of retry attempts before failing.
  timeout : int, default=REQUEST_TIMEOUT
      Timeout in seconds for each HTTP request.
  base_sleep : float, default=BASE_SLEEP
      Base number of seconds used for exponential backoff between retries.

  Returns
  -------
  dict
      Parsed JSON response from the server.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after all retry attempts.
  requests.exceptions.HTTPError
      If the server returns a 4xx/5xx response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * HTTP 429 triggers a wait using ``Retry-After`` if available, otherwise a fallback.
  * Logs warnings before each retry and errors if JSON parsing fails.
  * Uses a global ``requests.Session`` (`session`) for connection pooling.
  """

  for attempt in range(1, max_retries + 1):
    try:
      resp = session.get(url, params=params, timeout=timeout)
      
      if resp.status_code == 429:
        retry_after = resp.headers.get('Retry-After')
        if retry_after and retry_after.isdigit():
          sleep_for = int(retry_after)
        else:
          sleep_for = max(base_sleep * attempt, 15)
        logging.warning(f"HTTP 429 - wait {sleep_for}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      if 500 <= resp.status_code < 600:
        if attempt == max_retries:
          resp.raise_for_status()
        sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
        logging.warning(f"HTTP {resp.status_code} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
        time.sleep(sleep_for)
        continue

      resp.raise_for_status()

      try:
        return resp.json()
      except json.JSONDecodeError:
        logging.error('JSON parsing error')
        raise
    
    except requests.exceptions.RequestException as e:
      if attempt == max_retries:
        logging.exception('Request error (last attempt)')
        raise
      sleep_for = base_sleep * attempt + random.uniform(0, 1.0)
      logging.warning(f"{e} — retry in {sleep_for:.1f}s (attempt {attempt}/{max_retries})")
      time.sleep(sleep_for)

### Getting activities list

In [7]:
def fetch_all_activities(activites_url: str, per_page: int = PER_PAGE, max_pages: int = MAX_PAGES):
  """
  Fetch all activity records from a paginated API endpoint.

  The function iterates through API pages until either:
    * the maximum number of pages is reached (``max_pages``), or
    * the API returns an empty list (end of results).

  Each page is retrieved using ``get_json_with_retry`` to ensure resilience
  against transient errors (rate limits, timeouts, 5xx responses).

  Parameters
  ----------
  activites_url : str
      The base URL of the activities endpoint (must support ``per_page`` and ``page`` query params).
  per_page : int, default=PER_PAGE
      Number of activity records to request per page.
  max_pages : int, default=MAX_PAGES
      Maximum number of pages to fetch before stopping.

  Returns
  -------
  list of dict
      Combined list of activity objects returned by the API.

  Raises
  ------
  RuntimeError
      If the API response is not a list (unexpected schema).
  requests.exceptions.RequestException
      If the underlying HTTP requests fail after retries.

  Notes
  -----
  * Logs the number of records downloaded per page and the running total.
  * Stops early if the API returns an empty list before reaching ``max_pages``.
  * The total number of records is ``per_page * n_pages`` at most.
  """

  all_items = []
  page = 1
  while page <= max_pages:
    params = {'per_page': per_page, 'page': page}
    data = get_json_with_retry(activites_url, params=params)

    if not isinstance(data, list):
      raise RuntimeError(f"Unexpected response type for page {page}: {type(data)} — expected list")
    
    if not data:
      break
    
    all_items.extend(data)
    logging.info(f"Page {page}: downloaded {len(data)} records (total: {len(all_items)})")
    page += 1
  return all_items

activities_raw = fetch_all_activities(ACTIVITIES_URL, per_page=PER_PAGE, max_pages=MAX_PAGES)
logging.info(f"Total activities downloaded: {len(activities_raw)}")
activities_df = pd.json_normalize(activities_raw, sep='_')
if activities_df.empty:
    raise RuntimeError('No activities to save.')
activities_df.head()

2025-10-08 10:00:49,473 | INFO | Page 1: downloaded 200 records (total: 200)
2025-10-08 10:00:56,124 | INFO | Page 2: downloaded 200 records (total: 400)
2025-10-08 10:01:02,068 | INFO | Page 3: downloaded 200 records (total: 600)
2025-10-08 10:01:08,368 | INFO | Page 4: downloaded 200 records (total: 800)
2025-10-08 10:01:13,308 | INFO | Page 5: downloaded 200 records (total: 1000)
2025-10-08 10:01:14,483 | INFO | Page 6: downloaded 116 records (total: 1116)
2025-10-08 10:01:14,675 | INFO | Total activities downloaded: 1116


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete_id,athlete_resource_state,map_id,map_summary_polyline,map_resource_state,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Afternoon Weight Training,0.0,3616,3616,0.0,Workout,WeightTraining,NaN,16052008939,2025-10-06T13:00:37Z,2025-10-06T15:00:37Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,5,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.00,True,95.9,156.0,False,True,0.0,0.0,1.713449e+10,17134493018,garmin_ping_487976114519,False,0,0,False,8.0,81055898,1,a16052008939,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,13km Long Run⛰️,13061.6,4597,4749,139.0,Run,Run,2.0,16038944725,2025-10-05T08:24:17Z,2025-10-05T10:24:17Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,2,9,0,1,0,False,False,False,False,everyone,False,g24134620,"[52.75577, 15.22601]","[52.73852, 15.211718]",2.841,3.74,True,143.5,151.0,False,True,71.0,10.8,1.712123e+10,17121230328,garmin_ping_487547644497,False,2,0,False,49.0,81055898,1,a16038944725,ox~`Iuj|{ANjKAh@?jADVHFNEh@i@j@a@^@NGt@IbBLdC\...,2,84.5,367.6,546.0,361.0,True,1690.5,NaN
2,2,Evening Swim,2000.0,2343,2873,0.0,Swim,Swim,NaN,16033236291,2025-10-04T16:50:53Z,2025-10-04T18:50:53Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,5,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.854,1.80,True,127.8,147.0,False,True,0.0,0.0,1.711542e+10,17115415280,garmin_ping_487325437410,False,0,0,False,14.0,81055898,1,a16033236291,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Tempo 6km🔥,9416.9,2943,2943,17.0,Run,Run,3.0,16020263348,2025-10-03T12:44:47Z,2025-10-03T14:44:47Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,7,10,1,1,0,False,False,False,False,everyone,False,g23642256,"[51.108521, 17.120467]","[51.107243, 17.123941]",3.200,4.52,True,152.5,169.0,False,True,119.6,108.4,1.710227e+10,17102265794,garmin_ping_486913140510,False,7,0,False,57.0,81055898,1,a16020263348,_{}vHyvmgBo@|@cFjI_C~Cs@x@{@lA_@`@m@z@EN?PPl@z...,2,84.6,383.8,534.0,393.0,True,1129.7,NaN
4,2,9km Easy Run🌞,9120.3,3078,3078,13.0,Run,Run,NaN,16009596787,2025-10-02T12:40:09Z,2025-10-02T14:40:09Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,10,1,1,0,False,False,False,False,everyone,False,g24134620,"[51.107356, 17.124201]","[51.107426, 17.124272]",2.963,3.60,True,145.3,153.0,False,True,119.4,109.2,1.709146e+10,17091457144,garmin_ping_486552640886,False,0,0,False,40.0,81055898,1,a16009596787,qs{vHi~ngB\fALl@HPTX^jAdAbCl@jB\lBd@pDZdBJd@rA...,2,85.0,358.2,447.0,357.0,True,1102.7,NaN


### Set `bronze.activities` types map

In [8]:
activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

### Save `bronze.activities` to database

In [9]:
table_full_name = f"{TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"
logging.warning(f"Whole table {table_full_name} will be overwritten.")
activities_df.to_sql(
  ACTIVITIES_B_TABLE, 
  engine, 
  schema=TARGET_B_SCHEMA, 
  if_exists="replace", 
  index=False, 
  method="multi", 
  chunksize=1000, 
  dtype=activities_dtype_map
)

2025-10-08 10:01:14,760 | WARNING | Whole table bronze.activities will be overwritten.


-2

### Identify missing details — only `id` that are not present in `bronze.activities_details`

In [10]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  existing_ids = pd.read_sql(text(f"SELECT id FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)
  existing_ids_set = set(existing_ids['id'].astype('Int64').dropna().to_list())

all_ids_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids = sorted(all_ids_set - existing_ids_set)
logging.info(f"Missing details of {len(missing_ids)} IDs")
pd.DataFrame({'id': missing_ids}).head()

2025-10-08 10:01:15,700 | INFO | Missing details of 1 IDs


,id
0,16052008939


### Identify recent activities — update and download kudos of `id` from the last 30 days

In [11]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} (id BIGINT PRIMARY KEY)
"""))
  bronze_activities_df = pd.read_sql(text(f"SELECT * FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

In [12]:
bronze_activities_df['date'] =  pd.to_datetime(bronze_activities_df['start_date_local']).dt.tz_localize(None)
today = pd.Timestamp('today').normalize()
bronze_activities_df['activity_age_days'] = (today - bronze_activities_df['date'].dt.normalize()).dt.days

In [13]:
recent_activities_df = bronze_activities_df[bronze_activities_df['activity_age_days'] <= REFRESH_THRESHOLD_DAYS]
recent_activities_ids = set(recent_activities_df['id'].astype('Int64').dropna().to_list())
recent_activities_ids = sorted(recent_activities_ids)
logging.info(f"Refreshing details of {len(recent_activities_ids)} IDs")
pd.DataFrame({'id': recent_activities_ids}).head()

2025-10-08 10:01:16,055 | INFO | Refreshing details of 30 IDs


,id
0,15760949041
1,15766076458
2,15775473094
3,15786538213
4,15798063578


### Download activities details

In [14]:
def fetch_activity_details(activity_id: int):
  """
  Fetch detailed information for a single activity.

  Builds the activity detail endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity details as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_DETAIL_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_DETAIL_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

ids_to_be_downloaded = sorted(set(missing_ids) | set(recent_activities_ids))
details_records = []
for i, act_id in tqdm(enumerate(ids_to_be_downloaded, start=1), total=len(ids_to_be_downloaded)):
  try:
    resp = fetch_activity_details(act_id)
    
    if not isinstance(resp, dict):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue
    details_records.append(resp)
    
  except Exception as e:
    logging.error(f"Error downloading details for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Details downloaded: {len(details_records)} / {len(ids_to_be_downloaded)}")
details_df_new = pd.json_normalize(details_records, sep='_')

100%|██████████| 31/31 [04:32<00:00,  8.79s/it]
2025-10-08 10:05:48,650 | INFO | Details downloaded: 31 / 31


In [15]:
if not details_df_new.empty:
  details_df_new = details_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  details_df_new = details_df_new.dropna(how='all')
  
details_df_new.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete_id,athlete_resource_state,map_id,map_polyline,map_resource_state,map_summary_polyline,photos_primary,photos_count,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,splits_metric,splits_standard,best_efforts,gear_primary,gear_name,gear_nickname,gear_resource_state,gear_retired,gear_distance,gear_converted_distance,similar_activities_effort_count,similar_activities_average_speed,similar_activities_min_average_speed,similar_activities_mid_average_speed,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state,average_temp,photos_primary_unique_id,photos_primary_urls_600,photos_primary_urls_100,photos_primary_source,photos_primary_media_type,photos_use_primary_photo
0,3,Morning Weight Training,0.0,3625,3625,0.0,Workout,WeightTraining,NaN,15760949041,2025-09-10T07:01:39Z,2025-09-10T09:01:39Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,7,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.00,True,91.4,129.0,False,True,0.0,0.0,16835266098,16835266098,garmin_ping_478714840961,False,0,0,False,7.0,Reska8️⃣6️⃣,238.0,None,None,[],"[{'id': 56125613079, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,9934eae3e2f566aa7f7ad2f7dac9e885e616fc73,[heartrate],81055898,1,a15760949041,,3,,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,7.5km Easy Run😌,7556.8,2651,2651,11.0,Run,Run,0.0,15766076458,2025-09-10T15:44:58Z,2025-09-10T17:44:58Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,10,0,1,0,False,False,False,False,everyone,False,g24134620,"[51.107271, 17.124]","[51.103853, 17.120931]",2.851,3.58,True,144.3,156.0,False,True,124.8,116.2,16840645258,16840645258,garmin_ping_478851416260,False,0,0,False,30.0,7.5km Easy Run with Runna ✅\n\nLuźne klepanie🫡...,582.0,None,None,"[{'id': 3400905546860626570, 'resource_state':...","[{'id': 56144065886, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,2814aff65652a4fbab20db29c347c5a308e79cc8,"[heartrate, pace, power]",81055898,1,a15766076458,m{|vH}oogBHRD\Vb@Ph@HPLBZa@LINHXb@HBFATSVc@l@e...,3,k_|vHcpogB^v@b@dAJNNHT@n@Q\GTARBRHPRd@r@FRBv@J...,NaN,0,83.8,349.9,456.0,348.0,True,927.8,"[{'distance': 1000.9, 'elapsed_time': 353, 'el...","[{'distance': 1610.0, 'elapsed_time': 561, 'el...","[{'id': 66139861602, 'resource_state': 2, 'nam...",False,ASICS Novablast 5,None,2.0,False,415689.0,415.7,20.0,2.707158,1.858722,2.334599,3.138000,NaN,NaN,"[2.7660581928959, 2.893624189730874, 2.8697765...",3.0,1.858722,2.334599,3.138000,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Afternoon Weight Training,0.0,3576,3576,0.0,Workout,WeightTraining,NaN,15775473094,2025-09-11T13:01:52Z,2025-09-11T15:01:52Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,9,2,1,0,True,False,False,False,followers_only,Fa

### Set `bronze.activities_details` types map

In [16]:
activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
    "photos_primary_unique_id": Text,
    "photos_primary_urls_600": Text,
    "photos_primary_urls_100": Text,
    "photos_primary_source": Integer,
    "photos_primary_media_type": Integer,
    "photos_use_primary_photo": Boolean,
    "private_note": Text
}

### Save `bronze.activities_details` to database

In [17]:
if details_df_new.empty:
  logging.info('No new details to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA):
    details_df_new.head(0).to_sql(
      DETAILS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_details_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=DETAILS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE details_stg
        AS TABLE {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    details_df_new.to_sql(
      "details_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_details_dtype_map
      )
    
    # Prepare upsert
    stg_cols = details_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in details_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{DETAILS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM details_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS details_stg;'))
  
    logging.info('Activities details saved to PostgreSQL.')

2025-10-08 10:05:48,855 | INFO | Activities details saved to PostgreSQL.


### Sanity check

In [18]:
with engine.begin() as conn:
    total_acts = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}"), conn)
    total_det = pd.read_sql(text(f"SELECT COUNT(*) AS n FROM {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}"), conn)

logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{ACTIVITIES_B_TABLE}: {int(total_acts['n'][0])}")
logging.info(f"Total number of records in {TARGET_B_SCHEMA}.{DETAILS_B_TABLE}: {int(total_det['n'][0])}")

2025-10-08 10:05:48,880 | INFO | Total number of records in bronze.activities: 1116
2025-10-08 10:05:48,881 | INFO | Total number of records in bronze.activities_details: 1116


### Identify missing kudos — only `activity_id` that are not present in `bronze.kudos`

In [19]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_kudos = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}"), conn)
  existing_ids_kudos_set = set(existing_ids_kudos['activity_id'].astype('Int64').dropna().to_list())

all_ids_kudos_set = set(activities_df[activities_df['kudos_count'] > 0]['id'].astype('Int64').dropna().to_list())
missing_ids_kudos = sorted(all_ids_kudos_set - existing_ids_kudos_set, reverse=True)
logging.info(f"Missing kudos of {len(missing_ids_kudos)} activities")
pd.DataFrame({'activity_id': missing_ids_kudos}).head()

2025-10-08 10:05:48,903 | INFO | Missing kudos of 1 activities


,activity_id
0,16052008939


### Download activities kudos

In [20]:
def fetch_activity_kudos(activity_id: int):
  """
  Fetch all kudos for a single activity.

  Builds the activity kudos endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity kudos as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_KUDOS_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_KUDOS_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)

kudos_to_be_downloaded = sorted(set(missing_ids_kudos) | set(recent_activities_ids))

kudos_records = []
for i, act_id in tqdm(enumerate(kudos_to_be_downloaded, start=1), total=len(kudos_to_be_downloaded)):
  try:
    resp = fetch_activity_kudos(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    for i, kudos in enumerate(resp):
      kudos["activity_id"] = act_id
      kudos["kudos_id"] = i
    kudos_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading kudos for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"Kudos downloaded: {len(kudos_records)} / {len(kudos_to_be_downloaded)}")


100%|██████████| 31/31 [04:23<00:00,  8.49s/it]
2025-10-08 10:10:12,017 | INFO | Kudos downloaded: 286 / 31


In [21]:

kudos_df_new = pd.json_normalize(kudos_records, sep='_')
if not kudos_df_new.empty:
  kudos_df_new = kudos_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  kudos_df_new = kudos_df_new.dropna(how='all')
  
kudos_df_new['id'] = kudos_df_new[['activity_id', 'kudos_id']].astype("string").agg("-".join, axis=1)

In [22]:
kudos_df_new.head()

,resource_state,firstname,lastname,activity_id,kudos_id,id
0,2,Ola,Ł.,15760949041,0,15760949041-0
1,2,Mal,C.,15760949041,1,15760949041-1
2,2,Filip,C.,15760949041,2,15760949041-2
3,2,Agnieszka,G.,15760949041,3,15760949041-3
4,2,Karolina,C.,15760949041,4,15760949041-4


### Set `bronze.kudos` types map

In [23]:
activities_kudos_dtype_map = {
    "resource_state": Integer,
    "first_name": Text,
    "last_name": Float,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.kudos` to database

In [24]:
if kudos_df_new.empty:
  logging.info('No new kudos to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA):
    kudos_df_new.head(0).to_sql(
      KUDOS_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_kudos_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=KUDOS_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE kudos_stg
        AS TABLE {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    kudos_df_new.to_sql(
      "kudos_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_kudos_dtype_map
      )
    
    # Prepare upsert
    stg_cols = kudos_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in kudos_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{KUDOS_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM kudos_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS kudos_stg;'))
  
    logging.info('Activities kudos saved to PostgreSQL.')

2025-10-08 10:10:12,208 | INFO | Activities kudos saved to PostgreSQL.


### Identify missing zones — only `activity_id` that are not present in `bronze.activities_zones`

In [25]:
activities_df

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete_id,athlete_resource_state,map_id,map_summary_polyline,map_resource_state,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,average_temp
0,2,Afternoon Weight Training,0.0,3616,3616,0.0,Workout,WeightTraining,NaN,16052008939,2025-10-06T13:00:37Z,2025-10-06T15:00:37Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,5,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.00,True,95.9,156.0,False,True,0.0,0.0,1.713449e+10,17134493018,garmin_ping_487976114519,False,0,0,False,8.0,81055898,1,a16052008939,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,13km Long Run⛰️,13061.6,4597,4749,139.0,Run,Run,2.0,16038944725,2025-10-05T08:24:17Z,2025-10-05T10:24:17Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,2,9,0,1,0,False,False,False,False,everyone,False,g24134620,"[52.75577, 15.22601]","[52.73852, 15.211718]",2.841,3.74,True,143.5,151.0,False,True,71.0,10.8,1.712123e+10,17121230328,garmin_ping_487547644497,False,2,0,False,49.0,81055898,1,a16038944725,ox~`Iuj|{ANjKAh@?jADVHFNEh@i@j@a@^@NGt@IbBLdC\...,2,84.5,367.6,546.0,361.0,True,1690.5,NaN
2,2,Evening Swim,2000.0,2343,2873,0.0,Swim,Swim,NaN,16033236291,2025-10-04T16:50:53Z,2025-10-04T18:50:53Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,5,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.854,1.80,True,127.8,147.0,False,True,0.0,0.0,1.711542e+10,17115415280,garmin_ping_487325437410,False,0,0,False,14.0,81055898,1,a16033236291,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Tempo 6km🔥,9416.9,2943,2943,17.0,Run,Run,3.0,16020263348,2025-10-03T12:44:47Z,2025-10-03T14:44:47Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,7,10,1,1,0,False,False,False,False,everyone,False,g23642256,"[51.108521, 17.120467]","[51.107243, 17.123941]",3.200,4.52,True,152.5,169.0,False,True,119.6,108.4,1.710227e+10,17102265794,garmin_ping_486913140510,False,7,0,False,57.0,81055898,1,a16020263348,_{}vHyvmgBo@|@cFjI_C~Cs@x@{@lA_@`@m@z@EN?PPl@z...,2,84.6,383.8,534.0,393.0,True,1129.7,NaN
4,2,9km Easy Run🌞,9120.3,3078,3078,13.0,Run,Run,NaN,16009596787,2025-10-02T12:40:09Z,2025-10-02T14:40:09Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,10,1,1,0,False,False,False,False,everyone,False,g24134620,"[51.107356, 17.124201]","[51.107426, 17.124272]",2.963,3.60,True,145.3,153.0,False,True,119.4,109.2,1.709146e+10,17091457144,garmin_ping_486552640886,False,0,0,False,40.0,81055898,1,a16009596787,qs{vHi~ngB\fALl@HPTX^jAdAbCl@jB\lBd@pDZdBJd@rA...,2,85.0,358.2,447.0,357.0,True,1102.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,2,Morning Run,5640.4,2086,2100,44.8,Run,Run,NaN,6012647108,2016-07-19T08:07:29Z,2016-07-19T10:07:29Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,1,0,0,1,0,False,False,False,False,followers_only,False,g9239745,"[52.758578, 15.235688]","[52.758743, 15.235118]",2.704,3.70,False,NaN,NaN,False,False,60.9,38.1,6.390576e+09,6390575750,2016-07-19-100729.gpx,False,1,0,False,NaN,81055898,1,a6012647108,al_aI_v~{AA\e@tA]t@]j@aAbC]l@s@fBUr@]n@QNOVi@d...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1112,2,Morning Run,5067.5,1859,1859,36.7,Run,Run,NaN,6012647093,2016-07-17T07:43:59Z,2016-07-17T09:43:59Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,0,0,0,1,0,False,False,False,False,

In [26]:
with engine.begin() as conn:
  conn.execute(text(f"""
    CREATE TABLE IF NOT EXISTS {TARGET_B_SCHEMA}.{ZONES_B_TABLE} (id TEXT PRIMARY KEY, activity_id BIGINT)
"""))
  existing_ids_zones = pd.read_sql(text(f"SELECT activity_id FROM {TARGET_B_SCHEMA}.{ZONES_B_TABLE}"), conn)
  existing_ids_zones_set = set(existing_ids_zones['activity_id'].astype('Int64').dropna().to_list())

all_ids_zones_set = set(activities_df['id'].astype('Int64').dropna().to_list())
missing_ids_zones = sorted(all_ids_zones_set - existing_ids_zones_set, reverse=True)
logging.info(f"Missing zones of {len(missing_ids_zones)} activities")
pd.DataFrame({'activity_id': missing_ids_zones}).head()

2025-10-08 10:10:12,252 | INFO | Missing zones of 1 activities


,activity_id
0,16052008939


### Download activities zones

In [27]:
def fetch_activity_zones(activity_id: int):
  """
  Fetch all zones for a single activity.

  Builds the activity zones endpoint URL using the given activity ID
  and retrieves the JSON payload with retry logic.

  Parameters
  ----------
  activity_id : int
      Unique identifier of the activity.

  Returns
  -------
  dict
      JSON object containing the activity zones as returned by the API.

  Raises
  ------
  requests.exceptions.RequestException
      If the request fails after retries (e.g., network error, rate limit, 5xx).
  requests.exceptions.HTTPError
      If the server returns an error response on the last attempt.
  json.JSONDecodeError
      If the response cannot be parsed as JSON.

  Notes
  -----
  * Uses the global template ``ACTIVITY_ZONES_URL_TMPL`` to construct the URL.
  * Under the hood calls ``get_json_with_retry`` for resiliency.
  """
  url = ACTIVITY_ZONES_URL_TMPL.format(id=activity_id)
  return get_json_with_retry(url, params=None)


zones_records = []
for i, act_id in tqdm(enumerate(missing_ids_zones, start=1), total=len(missing_ids_zones)):
  try:
    resp = fetch_activity_zones(act_id)
    
    if not isinstance(resp, list):
      logging.warning(f"id={act_id}: unexpected response type ({type(resp)}), skip")
      continue

    if not resp:
      resp = [{'score': None,
              'distribution_buckets': None,
              'type': 'dummy',
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

    for i, zones in enumerate(resp):
      zones["activity_id"] = act_id
    zones_records.extend(resp)
    
  except Exception as e:
    logging.error(f"Error downloading zones for id={act_id}: {e}")
  
  time.sleep(random.randint(7, 9))

logging.info(f"zones downloaded: {len(zones_records)} / {len(missing_ids_zones)}")

100%|██████████| 1/1 [00:08<00:00,  8.37s/it]
2025-10-08 10:10:20,627 | INFO | zones downloaded: 1 / 1


In [28]:
resp = fetch_activity_zones(15191012381)

In [29]:
resp_test = [{'score': None,
              'distribution_buckets': None,
              'type': None,
              'resource_state': None,
              'sensor_based': None,
              'points': None,
              'custom_zones': None}]

In [30]:
zones_df_new = pd.json_normalize(zones_records, sep='_')
if not zones_df_new.empty:
  zones_df_new = zones_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  zones_df_new = zones_df_new.dropna(how='all')
  
zones_df_new['id'] = zones_df_new[['activity_id', 'type']].astype("string").agg("-".join, axis=1)

In [31]:
zones_df_new

,score,distribution_buckets,type,resource_state,sensor_based,points,custom_zones,activity_id,id
0,8.0,"[{'min': 0, 'max': 133, 'time': 3533.0}, {'min...",heartrate,3,True,0,True,16052008939,16052008939-heartrate


In [32]:
zones_records

[{'score': 8.0,
  'distribution_buckets': [{'min': 0, 'max': 133, 'time': 3533.0},
   {'min': 134, 'max': 147, 'time': 70.0},
   {'min': 148, 'max': 160, 'time': 13.0},
   {'min': 161, 'max': 166, 'time': 0.0},
   {'min': 167, 'max': -1, 'time': 0.0}],
  'type': 'heartrate',
  'resource_state': 3,
  'sensor_based': True,
  'points': 0,
  'custom_zones': True,
  'activity_id': 16052008939}]

In [33]:

zones_df_new = pd.json_normalize(zones_records, sep='_')
if not zones_df_new.empty:
  zones_df_new = zones_df_new.drop(['message', 'errors'], axis=1, errors='ignore')
  zones_df_new = zones_df_new.dropna(how='all')
  
zones_df_new['id'] = zones_df_new[['activity_id', 'type']].astype("string").agg("-".join, axis=1)

In [34]:
zones_df_new.head()

,score,distribution_buckets,type,resource_state,sensor_based,points,custom_zones,activity_id,id
0,8.0,"[{'min': 0, 'max': 133, 'time': 3533.0}, {'min...",heartrate,3,True,0,True,16052008939,16052008939-heartrate


### Set `bronze.activities_zones` types map

In [35]:
activities_zones_dtype_map = {
    "score": Float,
    "distribution_buckets": JSONB,
    "type": Text,
    "resource_state": Text,
    "sensor_based": Boolean,
    "points": Float,
    "custom_zones": Boolean,
    "activity_id": BigInteger,
    "id": Text
}

### Save `bronze.activities_zones` to database

In [36]:
if zones_df_new.empty:
  logging.info('No new kudos to be saved.')
else:
  insp = inspect(engine)

  # 1) Check if table exists in PostgreSQL
  if not insp.has_table(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA):
    zones_df_new.head(0).to_sql(
      ZONES_B_TABLE, 
      engine, 
      schema=TARGET_B_SCHEMA,
      if_exists="append", 
      index=False, 
      dtype=activities_zones_dtype_map
    )

    # 2) Check if primary key exists
  insp = inspect(engine)
  pk = insp.get_pk_constraint(table_name=ZONES_B_TABLE, schema=TARGET_B_SCHEMA)
  if not pk.get("constrained_columns"):
    with engine.begin() as conn:
      conn.execute(text(f'''
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ALTER COLUMN "id" SET NOT NULL;
        ALTER TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE}
        ADD PRIMARY KEY ("id");
      '''
      ))
    
    # 3) Staging
  with engine.begin() as conn:

    #Temp table
    conn.execute(text(f'''
        CREATE TEMP TABLE zones_stg
        AS TABLE {TARGET_B_SCHEMA}.{ZONES_B_TABLE} WITH NO DATA
      '''
      ))
    
    # Load data to temp table
    zones_df_new.to_sql(
      "zones_stg", 
      conn, 
      if_exists="append",
      index=False, 
      method="multi", 
      chunksize=5000, 
      dtype=activities_zones_dtype_map
      )
    
    # Prepare upsert
    stg_cols = zones_df_new.columns.to_list()
    
    if not 'id' in stg_cols:
      raise RuntimeError('Column "id" is required in zones_df_new to perform UPSERT.')
    
    cols_csv = ', '.join(f'"{c}"' for c in stg_cols)
    set_csv  = ', '.join(f'"{c}" = EXCLUDED."{c}"' for c in stg_cols if c != 'id')

    upsert_sql = f'''
        INSERT INTO {TARGET_B_SCHEMA}.{ZONES_B_TABLE} ({cols_csv})
        SELECT {cols_csv} FROM zones_stg
        ON CONFLICT ("id") DO UPDATE
        SET {set_csv};
      '''
    conn.execute(text(upsert_sql))

    # Clean up
    conn.execute(text('DROP TABLE IF EXISTS zones_stg;'))
  
    logging.info('Activities kudos saved to PostgreSQL.')

2025-10-08 10:10:20,992 | INFO | Activities kudos saved to PostgreSQL.
